rondamly select 1000 pieces of raw data to find the top 25% of bigram with highest frequency

In [1]:
import pandas as pd
import numpy as np
import pymysql

In [2]:
conn = pymysql.connect(host='localhost', user='root', password='Lmq19960916.', database='Twitter')
sqlcmd = "select* from clean2_train" 
raw = pd.read_sql(sqlcmd, conn)
conn.close()

In [3]:
def concat_func(x):
    return pd.Series({
        'text':' '.join(x['text']),
    })
data_com = raw.groupby([raw['ticker'],raw['created_time']]).apply(concat_func).reset_index()

In [4]:
print(data_com.head())
print(data_com.shape)

  ticker created_time                                               text
0    $AA   2016-12-31  calculated day expect return aa bay rating aa ...
1    $AA   2017-01-01  wonder risk aa day calculated day expect retur...
2    $AA   2017-01-02  corp aa lift buy inc corp aa lift buy inc corp...
3    $AA   2017-01-03  aa close mine significant fourth quarter corpo...
4    $AA   2017-01-04  bargain stock buy right stock aa power ion ene...
(1137114, 3)


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import time

In [6]:
cv = CountVectorizer(min_df=0.,max_df=1., ngram_range = (2,2))
tt = TfidfTransformer(norm='l2', use_idf=True)

In [7]:
l = []

for i in range(2000):
    data_sample = data_com.sample(n=2000, replace = False)
    cv_matrix = cv.fit_transform(data_sample.text).toarray()
    tt_matrix = tt.fit_transform(cv_matrix).toarray()
    vocab = cv.get_feature_names()
    sample_bi = pd.DataFrame(np.round(tt_matrix,2),columns = vocab)
    result_t = sample_bi.T 
    result_t['mean_value'] = result_t.mean(axis = 1)
    percentile75 = result_t.mean_value.describe()[6]
    df_try = result_t.drop(result_t[(result_t.mean_value < percentile75)].index)
    l.append(df_try.index)


In [8]:
flat_list = [item for sublist in l for item in sublist]

In [9]:
len(flat_list)

27850595

In [10]:
len(set(flat_list))

1241295

In [11]:
counts = dict()
for word in flat_list:
    counts[word] = counts.get(word,0) + 1

In [12]:
frame = pd.DataFrame(counts.items(), columns=['bigram', 'counts'])
frame.sort_values("counts",ascending=False, inplace=True)
frame

,bigram,counts
8459,scan result,2000
5662,llc million,2000
2947,earning thursday,2000
2945,earning stock,2000
2942,earning result,2000
...,...,...
1024154,accounting service,1
1024155,accounting services,1
1024156,ace bat,1
1024158,acid plasma,1


In [13]:
frame.counts.describe()

count    1.241295e+06
mean     2.243673e+01
std      1.124058e+02
min      1.000000e+00
25%      2.000000e+00
50%      4.000000e+00
75%      1.000000e+01
max      2.000000e+03
Name: counts, dtype: float64

In [14]:
percentile75 = frame.counts.describe()[6]
df_try = frame.drop(frame[(frame.counts <= 100)].index)
df_try

,bigram,counts
8459,scan result,2000
5662,llc million,2000
2947,earning thursday,2000
2945,earning stock,2000
2942,earning result,2000
...,...,...
205567,confirm pic,101
62747,contrast hotel,101
21718,tomorrow look,101
218214,analyst hawaiian,101


In [15]:
df_try.bigram.to_csv('bigram.csv',index = False)